# Weathering the Rails: Federal Railroad Administration Accident/Incident Data API
**Author:** Nathan Schaaf<br>
**Date:** December 10, 2024<br>
**Course:** Advanced Business Analytics, The Univerisity of North Carolina at Charlotte<br>
**Professional Context:** Prepared for the U.S. Railroad Industry (with focus on safety improvements)

## How to Use This Notebook
<ol>
<li>Prerequisites:</li>
<ul>
<li>Install the required Python libraries: pandas, requests, and datetime.</li>
<li>Adjust the url variable to the endpoint URL for the FRA dataset.</li>
</ul>
<li>Adjust Parameters:</li>
<ul>
<li>Set the limit variable to define the number of rows per request (default is 1,000).</li>
<li>The offset variable controls the starting point for each batch. The script automatically increments this value.</li>
</ul>
<li>Run the Script:</li>
<ul>
<li>Execute the script to fetch data in batches. Each batch will append data to a list until the API returns an empty response, signaling that all available records have been retrieved.</li>
<li>Depending on the size of the dataset and your connection speed, the script may take several minutes to complete.</li>
</ul>
<li>Save the Data:</li>
<ul>
<li>The data is stored in a pandas DataFrame, which you can save to a CSV file or use directly in further analysis.</li>
<li>Uncomment the print statements to display the total number of records retrieved and preview the first few rows.</li>
</ul>
<li>Caveats and Considerations:</li>
<ul>
<li>The API may occasionally fail or return incomplete data. If an error occurs, note the offset where the failure happened and restart the script from that point.</li>
<li>This script is designed for large-scale data retrieval, so ensure sufficient system memory is available for handling large datasets.</li>
</ol>
<br>
<br>
<strong>Conclusion</strong>
<p>By following these instructions, you can efficiently pull large amounts of FRA safety data for analysis or research purposes. This script offers flexibility to adapt to different dataset sizes while maintaining compliance with API limitations. <strong>Note</strong> this code will produce a dataset for the <strong>past 10 years</strong> from today.</p>

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [3]:
# WARNING can take up to 6 minutes to download.
# API URL
url = "https://data.transportation.gov/resource/85tf-25kj.json"

# Set the parameters
limit = 1000  # The number of rows to fetch per request
offset = 0    # The starting point for the next batch of rows
all_data = [] # To store all the data

while True:
    # Create the query string with the limit and offset
    query_url = f"{url}?$limit={limit}&$offset={offset}"
    
    # Make the API request
    response = requests.get(query_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Load the response into JSON format
        data = response.json()
        
        # If no data is returned, we've reached the end
        if not data:
            break
        
        # Append the data to our list
        all_data.extend(data)
        
        # Update the offset for the next batch of rows
        offset += limit
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        break

# Convert the list of records into a pandas DataFrame
df = pd.DataFrame(all_data)

# Display the number of rows fetched
#print(f"Total records retrieved: {len(df)}")

# Display the first few rows of the DataFrame
#print(df.head())

In [4]:
# Import primary accident cause codes related to track issues
from primaryAccidentCodesLibrary import primary_accident_cause_codes

# Get a list of track-related codes
codes_list = list(primary_accident_cause_codes.keys())

# Exclude rows with 'primaryaccidentcausecode' in the track-related codes list
non_track_accidents_df = df[~df['primaryaccidentcausecode'].isin(codes_list)].copy()

# Select relevant features for analysis
features_to_analyze = ['reportingrailroadcode', 'accidentnumber', 'date', 'time', 'accidenttype', 'hazmatreleasedcars', 'station', 'stateabbr', 'temperature', 'visibility_code', 'visibility', 'weathercondition', 'tracktype', 'equipmenttype', 'trainspeed', 'equipmentdamagecost', 'trackdamagecost', 'totaldamagecost', 'primaryaccidentcausecode', 'latitude', 'longitude']

# Drop all columns except the selected ones
non_track_accidents_df = non_track_accidents_df[features_to_analyze]

# Display the first few rows of the new DataFrame
#non_track_accidents_df.head()

In [5]:
# Ensure the 'date' column is in datetime format
non_track_accidents_df['date'] = pd.to_datetime(non_track_accidents_df['date'])

# Define the date 10 years ago from today
ten_years_ago = datetime.today() - timedelta(days=365*10)

# Filter the DataFrame to only include incidents from the past 10 years
non_track_accidents_df = non_track_accidents_df[non_track_accidents_df['date'] >= ten_years_ago]

# Check the new date range
date_min = non_track_accidents_df['date'].min()
date_max = non_track_accidents_df['date'].max()

print(f"The filtered incidents occurred between {date_min.date()} and {date_max.date()}.")

The filtered incidents occurred between 2014-11-16 and 2024-08-31.


In [6]:
# Export the filtered DataFrame to a CSV file
non_track_accidents_df.to_csv("non_track_accidents_last_10_years.csv", index=False)